<a href="https://colab.research.google.com/github/samuel-shaibu/NLP/blob/main/African_Tech_Fashion_Content_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install transformers
!pip install gradio==3.50.2 pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Suc

In [1]:
!pip install pytz

In [2]:
# Then, set up the app
from transformers import pipeline
import gradio as gr
import os
from datetime import datetime
from pyngrok import ngrok
import pytz

In [3]:
# Load a text generation pipeline
generator = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [39]:
history_file = "generated_history.txt"

def generate_content(prompt, task_type):

  try:
    if task_type != "Custom":
      prompt = f"{task_type}: {prompt}"

    result = generator(prompt, max_length=300, do_sample=True, temperature=0.9)
    output = result[0]['generated_text']


    # Save the result to file
    #local_time = datetime.utcnow() + timedelta(hours=1)
    naija_time = datetime.now(pytz.timezone("Africa/Lagos"))
    timestamp = naija_time.strftime("%Y-%m-%d %H:%M:%S")
    with open(history_file, "a") as f:
      f.write(f"\nTime: {timestamp}\nTask: {task_type}\nPrompt: {prompt}\nResponse: {output}\n{'-'*40}")

    return output

  except Exception as e:
    return f"⚠️ Error: {str(e)}"


In [40]:
def view_history():
  if os.path.exists(history_file):
    with open(history_file, "r") as f:
      return f.read()

  return "No history available yet."

In [41]:
def clear_history():
  if os.path.exists(history_file):
      open(history_file, "w").close() # wipe file contents
      return "History cleared."
  return "No history to clear."

In [42]:
# Gradio UI
task_types = ["Custom", "Poem", "Summary", "Article", "Story", "Code", "Chain-of-Thought"]


# Create tabs
with gr.Blocks() as demo:
  gr.Markdown(" ## ✨ African Tech + Fashion Content Generator")


  with gr.Tab("🧠 Content Generator"):
    with gr.Row():
      prompt_input = gr.Textbox(label="Enter your prompt or topic", lines=2)
      task_input = gr.Dropdown(task_types, label="Select task type", value="Custom")
    generate_button = gr.Button("Generate")
    output_box = gr.Textbox(label="Generated Output", lines =10)

    generate_button.click(
        generate_content,
        inputs=[prompt_input, task_input],
        outputs=output_box
    )

  with gr.Tab("📜 History Viewer"):
    history_button = gr.Textbox(label="History Log", lines=20)
    view_button = gr.Button("Load History")
    clear_button = gr.Button("Clear History")
    view_button.click(fn=view_history, outputs=history_button)
    clear_button.click(fn=clear_history, outputs=history_button)


In [37]:
# Start the Gradio app
port = 7861
demo.launch(server_name="0.0.0.0", server_port=port, share=True, inbrowser=False)

# Start ngrok tunner
public_url = ngrok.connect(port)
print(f"🔗 Your app is live at: {public_url}")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://45d44c29dc6e932a22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


🔗 Your app is live at: NgrokTunnel: "https://6be0-34-105-25-68.ngrok-free.app" -> "http://localhost:7861"


In [10]:
!ngrok config add-authtoken 2iEzQcEIiqoPKZuhJUhH6aE7d4d_7aXN71cdR1m5dwMzX3UCB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
